In [1]:

import cartopy
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
import pandas as pd

In [2]:
import os
print(os.listdir("../input"))

['glove6b100dtxt', 'd']


In [3]:
data=pd.read_csv('../input/d/datasets/omarmohamed25483/amazondatasetsaugmentedbyneutrals/DATASET - data - data (1).csv')
data.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [4]:
review=pd.DataFrame(data.groupby('reviews.rating').size().sort_values(ascending=False).rename('No of Users').reset_index())
review

,reviews.rating,No of Users
0,5.0,23775
1,-1.0,10413
2,1.0,9820
3,4.0,8541
4,3.0,1499
5,2.0,402


In [5]:
permanent = data[['reviews.rating' , 'reviews.text']]
mpermanent=permanent.dropna()

In [6]:
mpermanent['reviews.rating'].tail()

54478    1.0
54479    1.0
54480    1.0
54481    1.0
54482    1.0
Name: reviews.rating, dtype: float64

In [7]:
mpermanent['reviews.rating'].shape

(54448,)

In [8]:
# if rating >=3 label = 1
# if rating ==-1 label = 2
# if rating <3 label = 0
ratings = mpermanent['reviews.rating']
label = []
for i in ratings:
    if i >= 3:
        label.append('1')
    elif i == -1:
        label.append('2')
    else: 
        label.append('0')
        
mpermanent['label'] = label

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
reviews = mpermanent['reviews.text']
labels = mpermanent['label']

In [10]:
print(labels.shape)

(54448,)


In [11]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [12]:
def clean_document(doco):
    punctuation = string.punctuation
    punc_replace = ''.join([' ' for s in punctuation])
    doco_link_clean = re.sub(r'http\S+', '', doco)
    doco_clean_and = re.sub(r'&\S+', '', doco_link_clean)
    doco_clean_at = re.sub(r'@\S+', '', doco_clean_and)
    doco_clean = doco_clean_at.replace('-', ' ')
    doco_alphas = re.sub(r'\W +', ' ', doco_clean)
    trans_table = str.maketrans(punctuation, punc_replace)
    doco_clean = ' '.join([word.translate(trans_table) for word in doco_alphas.split(' ')])
    doco_clean = doco_clean.split(' ')
    p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
    doco_clean = [word.lower() for word in doco_clean if len(word) > 2]
    doco_clean = ([i for i in doco_clean if i not in stop])
#     doco_clean = [spell(word) for word in doco_clean]
#     p = re.compile(r'\s*\b(?=[a-z\d]*([a-z\d])\1{3}|\d+\b)[a-z\d]+', re.IGNORECASE)
    doco_clean = ([p.sub("", x).strip() for x in doco_clean])
#     doco_clean = ([spell(k) for k in doco_clean])
    return doco_clean

In [13]:
import regex as re
import string
review_cleans = [clean_document(doc) for doc in reviews];
sentences = [' '.join(r) for r in review_cleans ]

In [14]:
print(reviews[7])

I gave this as a Christmas gift to my inlaws, husband and uncle. They loved it and how easy they are to use with fantastic features!


In [15]:
print(sentences[7])

gave christmas gift inlaws husband uncle loved easy use fantastic features


In [16]:
MAX_SEQUENCE_LENGTH = 10
MAX_NUM_WORDS = 20000

In [17]:
# tiknization
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(sentences)
sequences = tokenizer.texts_to_sequences(sentences)
word_index = tokenizer.word_index # the dictionary
print('Found %s unique tokens.' % len(word_index)) #only top MAX_NUM_WORDS will be usedto generate the sequences
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of samples:', data.shape)
print('Sampele:(the zeros at the begining are for padding text to max length)')
print(data[2])

Found 37471 unique tokens.
Shape of samples: (54448, 10)
Sampele:(the zeros at the begining are for padding text to max length)
[ 646    2    5  536 1112 4695 1658  536 3728  253]


In [18]:
# Each label will be a binary vector of size 20
labels_matrix = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels_matrix.shape)
print('Sample label:\n',labels_matrix[1590])

Shape of data tensor: (54448, 10)
Shape of label tensor: (54448, 3)
Sample label:
 [0. 1. 0.]


In [19]:
VALIDATION_SPLIT = 0.2
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data_shuffled = data[indices]
labels_shuffled = labels_matrix[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data_shuffled.shape[0])
x_train = data_shuffled[:-nb_validation_samples]
y_train = labels_shuffled[:-nb_validation_samples]
x_val = data_shuffled[-nb_validation_samples:]
y_val = labels_shuffled[-nb_validation_samples:]
print('Shape of training data: ',x_train.shape)
print('Shape of testing data: ',x_val.shape)

Shape of training data:  (43559, 10)
Shape of testing data:  (10889, 10)


In [20]:
# Build a dictionary mapping words in the embeddings set to their embedding vector
EMBEDDING_DIM = 100
print('Indexing word vectors.')
embeddings_index = {}
with open('../input//glove6b100dtxt//glove.6B.100d.txt',encoding='UTF8') as f:
 for line in f:
     values = line.split(sep=' ')
     word = values[0]
     coefs = np.asarray(values[1:], dtype='float32')
     embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [21]:
# the words in the dataset that don't exist in Glove's dictionary will get a zeroes vector these words
# already have id 0 so there zeros embedding ector will be at index zero in the new matrix
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))#+1 to include the zerors vector for non-existing words
for word, i in word_index.items():
 embedding_vector = embeddings_index.get(word)
 if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
print ('Shape of Embedding Matrix: ',embedding_matrix.shape)


Shape of Embedding Matrix:  (37472, 100)


In [22]:
# responsible of converting a paded sequence of words IDs to a sequence of words embeddings
embedding_layer = Embedding(len(word_index) + 1, #vocab size
 EMBEDDING_DIM, #embedding vector size
 weights=[embedding_matrix], #weights matrix
 input_length=MAX_SEQUENCE_LENGTH, #padded sequence length
 trainable=False)

In [23]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 1, activation='relu')(embedded_sequences)
x = MaxPooling1D(1)(x)
x = Conv1D(128, 1, activation='relu')(x)
x = MaxPooling1D(1)(x)
x = Conv1D(128, 1, activation='relu')(x)
x = MaxPooling1D(10)(x) # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(3, activation='softmax')(x)

2022-05-31 05:12:57.860193: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [24]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
 optimizer='rmsprop',
 metrics=['acc'])
# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
 epochs=5, batch_size=128)

2022-05-31 05:12:58.146674: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
341/341 [==============================] - 6s 13ms/step - loss: 0.2507 - acc: 0.9118 - val_loss: 0.2741 - val_acc: 0.8985
Epoch 2/5
341/341 [==============================] - 5s 15ms/step - loss: 0.1486 - acc: 0.9512 - val_loss: 0.1595 - val_acc: 0.9497
Epoch 3/5
341/341 [==============================] - 4s 13ms/step - loss: 0.1208 - acc: 0.9609 - val_loss: 0.1558 - val_acc: 0.9496
Epoch 4/5
341/341 [==============================] - 5s 13ms/step - loss: 0.1043 - acc: 0.9658 - val_loss: 0.1500 - val_acc: 0.9544
Epoch 5/5
341/341 [==============================] - 5s 14ms/step - loss: 0.0930 - acc: 0.9695 - val_loss: 0.1795 - val_acc: 0.9432


In [25]:
print('Acuracy on testing set:')
model.evaluate(x_val,y_val)

Acuracy on testing set:
341/341 [==============================] - 1s 3ms/step - loss: 0.1795 - acc: 0.9432


[0.17948420345783234, 0.9432454705238342]

In [26]:
def my_function(predictionReview):
    sq = tokenizer.texts_to_sequences(predictionReview)
    predictionComment = pad_sequences(sq, maxlen=MAX_SEQUENCE_LENGTH)
    label_vec = model.predict(predictionComment.reshape(1,-1))
    max_index_row = np.argmax(label_vec)
    if max_index_row == 0 :
        print('negative')
    elif max_index_row == 1:
        print('postive')
    else:
        print('neutral')


In [27]:
# prediction part
# [0 1 0] for postive , [0 0 1] for neutral, [1 0 0 ] for negative
predictionReview = ['football player cristiano ronaldo scored the 5th goal of his career']
predictionReview1 = ['product always lags , inconsistent and have ton of bugs']
predictionReview2 = ['last year i bought this charger and it is one of the best chargers i have used ever']
my_function(predictionReview)
my_function(predictionReview1)
my_function(predictionReview2)

neutral
negative
postive
